In [70]:
# Dataset from: https://www.kaggle.com/litzar/fruits-classification
from os import walk

from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers

In [71]:
# Use data augmentation to create a larger training set
def create_augmented_samples(data_generator, source_dir, output_dir, prefix, num_samples=10):
    if num_samples < 1:
        return
    img = load_img(source_dir)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)
    i = 0
    cur_fruit = 'Apricot'
    for batch in datagen.flow(x, batch_size=1, save_to_dir='fruits-datagen',
                              save_prefix=prefix, save_format='jpeg'):
        i += 1
        if i > num_samples:
            break

In [72]:
training_dir = 'fruits-360/Training/'
fruit_names = [x[0].split('/')[-1] for x in walk(training_dir) if x[0].split('/')[-1] is not '']

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

for fruit in fruit_names:
    for y in [x[-1] for x in walk(training_dir+fruit)]:
        for z in y:
            training_ex_dir = training_dir+'/'.join([fruit, z])
            create_augmented_samples(data_generator=datagen,
                                     source_dir=training_ex_dir,
                                     output_dir='fruits-datagen/'+fruit,
                                     prefix=fruit,
                                     num_samples=0)

In [73]:
f = []
for (dirpath, dirnames, filenames) in walk('fruits-datagen'):
    f.extend(filenames)

In [74]:
model = Sequential()

# Convoluation Layers
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(data_format="channels_first", pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), data_format="channels_first"))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Fully Connected Layers
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(60, activation='softmax'))

#Optimizers + Compile
adam_opt = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=.001),
              metrics=['accuracy'])

In [75]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'fruits-360/Training',  # target directory
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        'fruits-360/Validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 28736 images belonging to 60 classes.
Found 9673 images belonging to 60 classes.


In [76]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('model_weights/fifty_epoch.h5')

Epoch 1/50
125/125 [==============================] - 154s 1s/step - loss: 4.0992 - acc: 0.0475 - val_loss: 3.6435 - val_acc: 0.1237
Epoch 2/50
125/125 [==============================] - 138s 1s/step - loss: 3.3027 - acc: 0.1530 - val_loss: 2.3824 - val_acc: 0.3650
Epoch 3/50
125/125 [==============================] - 142s 1s/step - loss: 2.5583 - acc: 0.2645 - val_loss: 1.7856 - val_acc: 0.4913
Epoch 4/50
125/125 [==============================] - 153s 1s/step - loss: 1.9884 - acc: 0.3980 - val_loss: 1.1345 - val_acc: 0.7338
Epoch 5/50
125/125 [==============================] - 147s 1s/step - loss: 1.6430 - acc: 0.5075 - val_loss: 0.8217 - val_acc: 0.7913
Epoch 6/50
125/125 [==============================] - 150s 1s/step - loss: 1.3428 - acc: 0.5830 - val_loss: 0.7125 - val_acc: 0.7863
Epoch 7/50
125/125 [==============================] - 172s 1s/step - loss: 1.1727 - acc: 0.6395 - val_loss: 0.6379 - val_acc: 0.7950
Epoch 8/50
125/125 [==============================] - 170s 1s/step - 

In [77]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_75 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 148, 74, 16)       0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 32, 72, 14)        42656     
_________________________________________________________________
activation_76 (Activation)   (None, 32, 72, 14)        0         
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 16, 36, 14)        0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 14, 34, 64)        8128      
__________